<a href="https://colab.research.google.com/github/debora28/pi-2020.2/blob/main/Crime_E2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Dataset sobre taxa de ocorrência de crimes como roubo, furto e relacionados
*Os dados foram extraídos do sistema de Registro Digital de Ocorrências (R.D.O.) disponível no site www.ssp.sp.gov.br referente ao ano de 2016.*

*Importando as bibliotecas necessárias para tratamento, estatística e visualização de dados:*

In [120]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

*Carregando o arquivo:*

In [121]:
# Diretamente do GitHub:
url = 'https://raw.githubusercontent.com/debora28/pi-2020.2/main/Edited_CrimeRatesBrazil.csv'
data = pd.read_csv(url)

In [122]:
# Lendo o arquivo CSV que foi carregado na página:
# data = pd.read_csv('Edited_CrimeRatesBrazil.csv')

# Siglas e termos:

*DECAP = Departamento de Polícia Judiciária da Capital;
Seccionais = Subdivisões da DECAP, por unidade territorial;
Flag_status = status de registro;
CIRC = Circunscrição;
Rubrica = Natureza jurídica da ocorrência, tipo de crime.
Status:
  "Remarcado" ('R')
  "Cancelado" ('C')
  "Convertido em Diligência" ('D')
  "Realizada" ('F')
  "Não realizada" ('N')
  Disponível apenas para usuários internos e para audiências ('M')*

#Limpeza e Tratamento
Procurando nulos e removendo: NA - Not Available e NaN - Not a Number.
Podemos ver que 5 colunas retornaram True, ou seja, possuem pelo menos um valor (any) indisponível ou vazio (isnull).

In [123]:
print(data.isnull().any())
print()

NUM_BO                    False
ANO_BO                    False
ID_DELEGACIA              False
NOME_DEPARTAMENTO         False
NOME_SECCIONAL            False
DELEGACIA                 False
NOME_DEPARTAMENTO_CIRC    False
NOME_SECCIONAL_CIRC       False
NOME_DELEGACIA_CIRC       False
ANO                       False
MES                       False
FLAG_STATUS               False
RUBRICA                   False
DESDOBRAMENTO              True
CONDUTA                    True
LATITUDE                   True
LONGITUDE                  True
CIDADE                    False
LOGRADOURO                False
NUMERO_LOGRADOURO         False
FLAG_STATUS.1             False
Unnamed: 21                True
dtype: bool



Aqui podemos ver que:
- A coluna DESDOBRAMENTO tem apenas 5 valores;
- As colunas CONDUTA, LATITUDE e LONGITUDE estão quase completas;
- Unnamed: 21 está vazia;
- FLAG_STATUS.1 é uma repetição de FLAG_STATUS.

In [124]:
print(data.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 199 entries, 0 to 198
Data columns (total 22 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   NUM_BO                  199 non-null    int64  
 1   ANO_BO                  199 non-null    int64  
 2   ID_DELEGACIA            199 non-null    int64  
 3   NOME_DEPARTAMENTO       199 non-null    object 
 4   NOME_SECCIONAL          199 non-null    object 
 5   DELEGACIA               199 non-null    object 
 6   NOME_DEPARTAMENTO_CIRC  199 non-null    object 
 7   NOME_SECCIONAL_CIRC     199 non-null    object 
 8   NOME_DELEGACIA_CIRC     199 non-null    object 
 9   ANO                     199 non-null    int64  
 10  MES                     199 non-null    int64  
 11  FLAG_STATUS             199 non-null    object 
 12  RUBRICA                 199 non-null    object 
 13  DESDOBRAMENTO           5 non-null      object 
 14  CONDUTA                 185 non-null    ob

Podemos então apagar as colunas DESDOBRAMENTO, Unnamed: 21 e FLAG_STATUS.1, por falta ou insuficiência de valores.

Com drop podemos excluir as colunas (axis=1 são colunas, axis=0 são linhas) sem utilidade, salvando em um novo dataframe (recebido pela variável data2) e depois salvando num outro arquivo csv (com to_csv):

In [125]:
data2 = data.drop('DESDOBRAMENTO', axis=1)
data2.to_csv('data_semDesdob.csv', index= False)
data2.head()

,NUM_BO,ANO_BO,ID_DELEGACIA,NOME_DEPARTAMENTO,NOME_SECCIONAL,DELEGACIA,NOME_DEPARTAMENTO_CIRC,NOME_SECCIONAL_CIRC,NOME_DELEGACIA_CIRC,ANO,MES,FLAG_STATUS,RUBRICA,CONDUTA,LATITUDE,LONGITUDE,CIDADE,LOGRADOURO,NUMERO_LOGRADOURO,FLAG_STATUS.1,Unnamed: 21
0,3784,2016,10101,DECAP,DEL.SEC.1CENTRO,01D.P.SE,DECAP,DEL.SEC.1CENTRO,78D.P.JARDINS,2016,7,C,Furto(art.155),TRANSEUNTE,-23.564984,46.652035,S.PAULO,AVENIDAPAULISTA,1000,C,NaN
1,3426,2016,10102,DECAP,DEL.SEC.1CENTRO,02D.P.BOMRETIRO,DECAP,DEL.SEC.1CENTRO,03D.P.CAMPOSELISEOS,2016,5,C,Roubo(art.157),TRANSEUNTE,-23.542476,-46.641928,S.PAULO,PRAADAREPUBLICA,0,C,NaN
2,6359,2016,10102,DECAP,DEL.SEC.1CENTRO,02D.P.BOMRETIRO,DECAP,DEL.SEC.1CENTRO,03D.P.CAMPOSELISEOS,2016,10,C,Drogassemautorizaoouemdesacordo(Art.33caput),NaN,-23.542183,-46.640599,S.PAULO,RUACONSELHEIRONEBIAS,0,C,NaN
3,1267,2016,10103,DECAP,DEL.SEC.1CENTRO,03D.P.CAMPOSELISEOS,DECAP,DEL.SEC.8SAOMATEUS,49D.P.SAOMATEUS,2016,3,C,Roubo(art.157),CARGA,-23.609275,-46.455087,S.PAULO,RUAMADUREIRACALHEIROS,15,C,NaN
4,4804,2016,10106,DECAP,DEL.SEC.1CENTRO,06D.P.CAMBUCI,DECAP,DEL.SEC.1CENTRO,06D.P.CAMBUCI,2016,12,C,Lesocorporal(art.129),NaN,-23.573928,-46.620693,S.PAULO,RUAROBERTSON,625,C,NaN


In [126]:
data2 = data2.drop('Unnamed: 21', axis=1)
data2.to_csv('data_semUnnamed.csv', index= False)
data2.head()

,NUM_BO,ANO_BO,ID_DELEGACIA,NOME_DEPARTAMENTO,NOME_SECCIONAL,DELEGACIA,NOME_DEPARTAMENTO_CIRC,NOME_SECCIONAL_CIRC,NOME_DELEGACIA_CIRC,ANO,MES,FLAG_STATUS,RUBRICA,CONDUTA,LATITUDE,LONGITUDE,CIDADE,LOGRADOURO,NUMERO_LOGRADOURO,FLAG_STATUS.1
0,3784,2016,10101,DECAP,DEL.SEC.1CENTRO,01D.P.SE,DECAP,DEL.SEC.1CENTRO,78D.P.JARDINS,2016,7,C,Furto(art.155),TRANSEUNTE,-23.564984,46.652035,S.PAULO,AVENIDAPAULISTA,1000,C
1,3426,2016,10102,DECAP,DEL.SEC.1CENTRO,02D.P.BOMRETIRO,DECAP,DEL.SEC.1CENTRO,03D.P.CAMPOSELISEOS,2016,5,C,Roubo(art.157),TRANSEUNTE,-23.542476,-46.641928,S.PAULO,PRAADAREPUBLICA,0,C
2,6359,2016,10102,DECAP,DEL.SEC.1CENTRO,02D.P.BOMRETIRO,DECAP,DEL.SEC.1CENTRO,03D.P.CAMPOSELISEOS,2016,10,C,Drogassemautorizaoouemdesacordo(Art.33caput),NaN,-23.542183,-46.640599,S.PAULO,RUACONSELHEIRONEBIAS,0,C
3,1267,2016,10103,DECAP,DEL.SEC.1CENTRO,03D.P.CAMPOSELISEOS,DECAP,DEL.SEC.8SAOMATEUS,49D.P.SAOMATEUS,2016,3,C,Roubo(art.157),CARGA,-23.609275,-46.455087,S.PAULO,RUAMADUREIRACALHEIROS,15,C
4,4804,2016,10106,DECAP,DEL.SEC.1CENTRO,06D.P.CAMBUCI,DECAP,DEL.SEC.1CENTRO,06D.P.CAMBUCI,2016,12,C,Lesocorporal(art.129),NaN,-23.573928,-46.620693,S.PAULO,RUAROBERTSON,625,C


In [127]:
data2 = data2.drop('FLAG_STATUS.1', axis=1)
data2.to_csv('data_semFlag1.csv', index= False)
data2.head()

,NUM_BO,ANO_BO,ID_DELEGACIA,NOME_DEPARTAMENTO,NOME_SECCIONAL,DELEGACIA,NOME_DEPARTAMENTO_CIRC,NOME_SECCIONAL_CIRC,NOME_DELEGACIA_CIRC,ANO,MES,FLAG_STATUS,RUBRICA,CONDUTA,LATITUDE,LONGITUDE,CIDADE,LOGRADOURO,NUMERO_LOGRADOURO
0,3784,2016,10101,DECAP,DEL.SEC.1CENTRO,01D.P.SE,DECAP,DEL.SEC.1CENTRO,78D.P.JARDINS,2016,7,C,Furto(art.155),TRANSEUNTE,-23.564984,46.652035,S.PAULO,AVENIDAPAULISTA,1000
1,3426,2016,10102,DECAP,DEL.SEC.1CENTRO,02D.P.BOMRETIRO,DECAP,DEL.SEC.1CENTRO,03D.P.CAMPOSELISEOS,2016,5,C,Roubo(art.157),TRANSEUNTE,-23.542476,-46.641928,S.PAULO,PRAADAREPUBLICA,0
2,6359,2016,10102,DECAP,DEL.SEC.1CENTRO,02D.P.BOMRETIRO,DECAP,DEL.SEC.1CENTRO,03D.P.CAMPOSELISEOS,2016,10,C,Drogassemautorizaoouemdesacordo(Art.33caput),NaN,-23.542183,-46.640599,S.PAULO,RUACONSELHEIRONEBIAS,0
3,1267,2016,10103,DECAP,DEL.SEC.1CENTRO,03D.P.CAMPOSELISEOS,DECAP,DEL.SEC.8SAOMATEUS,49D.P.SAOMATEUS,2016,3,C,Roubo(art.157),CARGA,-23.609275,-46.455087,S.PAULO,RUAMADUREIRACALHEIROS,15
4,4804,2016,10106,DECAP,DEL.SEC.1CENTRO,06D.P.CAMBUCI,DECAP,DEL.SEC.1CENTRO,06D.P.CAMBUCI,2016,12,C,Lesocorporal(art.129),NaN,-23.573928,-46.620693,S.PAULO,RUAROBERTSON,625


Observamos que a coluna mais importante para o nosso estudo é a Rubrica, a qual traz a natureza do crime. No entanto, devemos usar o número de ocorrências de cada crime, então vamos transformar as linhas dessa coluna em novas colunas. 
Em cada coluna nova, aparecerá 1 onde o título corresponder à coluna antiga, ou 0 se não houver correspondência:

In [128]:
novas_colunas = pd.get_dummies(data2['RUBRICA']) 
data2 = pd.concat([data2, novas_colunas], axis=1) 
# axis = 1 concatena colunas. axis = 0 concatena linhas
data2.head()

,NUM_BO,ANO_BO,ID_DELEGACIA,NOME_DEPARTAMENTO,NOME_SECCIONAL,DELEGACIA,NOME_DEPARTAMENTO_CIRC,NOME_SECCIONAL_CIRC,NOME_DELEGACIA_CIRC,ANO,MES,FLAG_STATUS,RUBRICA,CONDUTA,LATITUDE,LONGITUDE,CIDADE,LOGRADOURO,NUMERO_LOGRADOURO,A.I.-Lesocorporal(art.129),Drogassemautorizaoouemdesacordo(Art.33caput),Furto(art.155),Furtoqualificado(art.155_4o.),Lesocorporal(art.129),Lesocorporal(art1299),Lesocorporalculposanadireodeveculoautomotor(Art.303),Roubo(art.157)
0,3784,2016,10101,DECAP,DEL.SEC.1CENTRO,01D.P.SE,DECAP,DEL.SEC.1CENTRO,78D.P.JARDINS,2016,7,C,Furto(art.155),TRANSEUNTE,-23.564984,46.652035,S.PAULO,AVENIDAPAULISTA,1000,0,0,1,0,0,0,0,0
1,3426,2016,10102,DECAP,DEL.SEC.1CENTRO,02D.P.BOMRETIRO,DECAP,DEL.SEC.1CENTRO,03D.P.CAMPOSELISEOS,2016,5,C,Roubo(art.157),TRANSEUNTE,-23.542476,-46.641928,S.PAULO,PRAADAREPUBLICA,0,0,0,0,0,0,0,0,1
2,6359,2016,10102,DECAP,DEL.SEC.1CENTRO,02D.P.BOMRETIRO,DECAP,DEL.SEC.1CENTRO,03D.P.CAMPOSELISEOS,2016,10,C,Drogassemautorizaoouemdesacordo(Art.33caput),NaN,-23.542183,-46.640599,S.PAULO,RUACONSELHEIRONEBIAS,0,0,1,0,0,0,0,0,0
3,1267,2016,10103,DECAP,DEL.SEC.1CENTRO,03D.P.CAMPOSELISEOS,DECAP,DEL.SEC.8SAOMATEUS,49D.P.SAOMATEUS,2016,3,C,Roubo(art.157),CARGA,-23.609275,-46.455087,S.PAULO,RUAMADUREIRACALHEIROS,15,0,0,0,0,0,0,0,1
4,4804,2016,10106,DECAP,DEL.SEC.1CENTRO,06D.P.CAMBUCI,DECAP,DEL.SEC.1CENTRO,06D.P.CAMBUCI,2016,12,C,Lesocorporal(art.129),NaN,-23.573928,-46.620693,S.PAULO,RUAROBERTSON,625,0,0,0,0,1,0,0,0


# Analisando os dados
Com describe, vemos que apesar de ser referente ao ano de 2016, há pelo menos uma ocorrência de 2017 na coluna ANO_BO (última linha -> max), o que sugere que um crime ocorrido em 2016 só foi registrado em 2017. Não pudemos identificar nenhum outlier.

In [129]:
data2.describe()

,NUM_BO,ANO_BO,ID_DELEGACIA,ANO,MES,LATITUDE,LONGITUDE,NUMERO_LOGRADOURO,A.I.-Lesocorporal(art.129),Drogassemautorizaoouemdesacordo(Art.33caput),Furto(art.155),Furtoqualificado(art.155_4o.),Lesocorporal(art.129),Lesocorporal(art1299),Lesocorporalculposanadireodeveculoautomotor(Art.303),Roubo(art.157)
count,1.990000e+02,199.000000,199.000000,199.0,199.000000,188.000000,188.000000,199.000000,199.000000,199.000000,199.000000,199.000000,199.000000,199.000000,199.000000,199.000000
mean,4.687784e+05,2016.020101,500213.251256,2016.0,6.658291,-23.574926,-46.113215,688.437186,0.005025,0.010050,0.462312,0.075377,0.030151,0.015075,0.010050,0.391960
std,5.535074e+05,0.140698,437009.554279,0.0,3.374566,0.079832,6.802924,1437.826725,0.070888,0.099997,0.499835,0.264664,0.171433,0.122160,0.099997,0.489419
min,3.600000e+01,2016.000000,10004.000000,2016.0,1.000000,-23.808308,-46.958875,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,3.555500e+03,2016.000000,25201.500000,2016.0,4.000000,-23.618337,-46.687784,37.500000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,1.442190e+05,2016.000000,900020.000000,2016.0,7.000000,-23.559011,-46.623184,248.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,8.584790e+05,2016.000000,900020.000000,2016.0,10.000000,-23.528155,-46.531476,549.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,1.000000
max,1.649798e+06,2017.000000,900020.000000,2016.0,12.000000,-23.281891,46.652035,13913.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


Removendo a coluna RUBRICA, que já foi adcionada como outras colunas:

In [130]:
data2 = data2.drop('RUBRICA', axis=1)
data2.to_csv('data_novasCols.csv', index= False)
data2.head()

,NUM_BO,ANO_BO,ID_DELEGACIA,NOME_DEPARTAMENTO,NOME_SECCIONAL,DELEGACIA,NOME_DEPARTAMENTO_CIRC,NOME_SECCIONAL_CIRC,NOME_DELEGACIA_CIRC,ANO,MES,FLAG_STATUS,CONDUTA,LATITUDE,LONGITUDE,CIDADE,LOGRADOURO,NUMERO_LOGRADOURO,A.I.-Lesocorporal(art.129),Drogassemautorizaoouemdesacordo(Art.33caput),Furto(art.155),Furtoqualificado(art.155_4o.),Lesocorporal(art.129),Lesocorporal(art1299),Lesocorporalculposanadireodeveculoautomotor(Art.303),Roubo(art.157)
0,3784,2016,10101,DECAP,DEL.SEC.1CENTRO,01D.P.SE,DECAP,DEL.SEC.1CENTRO,78D.P.JARDINS,2016,7,C,TRANSEUNTE,-23.564984,46.652035,S.PAULO,AVENIDAPAULISTA,1000,0,0,1,0,0,0,0,0
1,3426,2016,10102,DECAP,DEL.SEC.1CENTRO,02D.P.BOMRETIRO,DECAP,DEL.SEC.1CENTRO,03D.P.CAMPOSELISEOS,2016,5,C,TRANSEUNTE,-23.542476,-46.641928,S.PAULO,PRAADAREPUBLICA,0,0,0,0,0,0,0,0,1
2,6359,2016,10102,DECAP,DEL.SEC.1CENTRO,02D.P.BOMRETIRO,DECAP,DEL.SEC.1CENTRO,03D.P.CAMPOSELISEOS,2016,10,C,NaN,-23.542183,-46.640599,S.PAULO,RUACONSELHEIRONEBIAS,0,0,1,0,0,0,0,0,0
3,1267,2016,10103,DECAP,DEL.SEC.1CENTRO,03D.P.CAMPOSELISEOS,DECAP,DEL.SEC.8SAOMATEUS,49D.P.SAOMATEUS,2016,3,C,CARGA,-23.609275,-46.455087,S.PAULO,RUAMADUREIRACALHEIROS,15,0,0,0,0,0,0,0,1
4,4804,2016,10106,DECAP,DEL.SEC.1CENTRO,06D.P.CAMBUCI,DECAP,DEL.SEC.1CENTRO,06D.P.CAMBUCI,2016,12,C,NaN,-23.573928,-46.620693,S.PAULO,RUAROBERTSON,625,0,0,0,0,1,0,0,0
